In [ ]:
print("ok")

ok


In [ ]:
from dotenv import load_dotenv

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
# Test Groq API with different models to find available ones
from dotenv import load_dotenv
import os
load_dotenv()

# List of models to test
models_to_test = [
    "llama-3.1-8b-instant",
    "llama-3.3-70b-versatile",
    "llama-3.2-90b-vision-preview",
    "llama-3.2-11b-vision-preview",
    "llama-3.2-3b",
    "llama-3.1-70b-versatile",
    "mixtral-8x7b-32768",
    "gemma-7b-it",
    "gemma2-9b-it",
    "llama-3.3-8b-instant",
]

print("Testing Groq models to find available ones...\n")
available_models = []
decommissioned_models = []

for model_name in models_to_test:
    try:
        print(f"Testing {model_name}...", end=" ")
        llm = ChatGroq(model=model_name, temperature=0)
        response = llm.invoke("Say 'OK' if you can hear me")
        print(f"✓ AVAILABLE")
        available_models.append(model_name)
    except Exception as e:
        error_msg = str(e)
        if "decommissioned" in error_msg.lower():
            print(f"✗ DECOMMISSIONED")
            decommissioned_models.append(model_name)
        else:
            print(f"✗ ERROR: {error_msg[:50]}...")

print(f"\n{'='*60}")
print(f"AVAILABLE MODELS: {len(available_models)}")
for model in available_models:
    print(f"  ✓ {model}")
    
print(f"\nDECOMMISSIONED MODELS: {len(decommissioned_models)}")
for model in decommissioned_models:
    print(f"  ✗ {model}")


In [ ]:
# Test a specific model (update model_name after running the test above)
# Use the first available model from the list above
if available_models:
    test_model = available_models[0]
    print(f"Testing {test_model} with a sample query...\n")
    
    llm = ChatGroq(model=test_model, temperature=0)
    response = llm.invoke("What is 2+2? Answer in one sentence.")
    print(f"Response: {response.content}")
    print(f"\n✓ Model {test_model} is working correctly!")
    print(f"\nRecommended model for config.yaml: {test_model}")
else:
    print("No available models found. Please check your GROQ_API_KEY or try different model names.")


: 

In [ ]:
# Optionally: Update config.yaml with the first available model
import yaml
import os


if available_models:
    recommended_model = available_models[0]
    config_path = "../config/config.yaml"
    
    # Read current config
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    # Update Groq model
    old_model = config['llm']['groq']['model_name']
    config['llm']['groq']['model_name'] = recommended_model
    
    # Write back
    with open(config_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    
    print(f"✓ Updated config.yaml:")
    print(f"  Old model: {old_model}")
    print(f"  New model: {recommended_model}")
    print(f"\nPlease restart your uvicorn server for changes to take effect.")
else:
    print("No available models to update config with.")


: 

In [ ]:
# Quick test: Test a single model (modify model_name as needed)
from dotenv import load_dotenv
load_dotenv()

# Change this to test different models
test_model_name = "llama-3.1-8b-instant"  # Try: llama-3.1-8b-instant, llama-3.3-70b-versatile, etc.

try:
    print(f"Testing model: {test_model_name}\n")
    llm = ChatGroq(model=test_model_name, temperature=0)
    response = llm.invoke("Hello! Please respond with 'Model is working' if you can hear me.")
    print(f"✓ SUCCESS!")
    print(f"Response: {response.content}")
    print(f"\nThis model is available and can be used in config.yaml")
except Exception as e:
    error_msg = str(e)
    if "decommissioned" in error_msg.lower():
        print(f"✗ Model {test_model_name} has been DECOMMISSIONED")
        print(f"Error: {error_msg}")
    else:
        print(f"✗ Error testing model: {error_msg}")


In [ ]:
llm.invoke("hi")

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 4, 'total_tokens': 20, 'completion_time': 0.084725173, 'prompt_time': 6.0159e-05, 'queue_time': 0.053524741, 'total_time': 0.084785332}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--8b091ea3-0a7d-4058-9e8b-2a68094cc078-0', usage_metadata={'input_tokens': 4, 'output_tokens': 16, 'total_tokens': 20})

In [ ]:
from langchain.tools import tool

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [ ]:
multiply

StructuredTool(name='multiply', description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002DFC40C3A30>)

In [ ]:
from langchain_core.tools import StructuredTool

In [ ]:
class WeatherInput(BaseModel):
    city: str

In [ ]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

In [ ]:
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fetches real-time weather data for a city",
    args_schema=WeatherInput,  
)

In [ ]:
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"           
    description: ClassVar[str] = (
        "Fetches weather data for a city"
    )
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput

    def _run(self, city: str, units: str = "metric") -> str:
        return get_weather(city, units)